1- How many tennis players are included in the dataset?

In [33]:
import pandas as pd
MatchHomeTeamInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchHomeTeamInfo.csv")
MatchAwayTeamInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchAwayTeamInfo.csv")

home_players = MatchHomeTeamInfo['player_id']
away_players = MatchAwayTeamInfo['player_id']
all_players = pd.concat([home_players, away_players]).unique()
num_unique_players = len(all_players)
print(f'Total unique tennis players: {num_unique_players}')

Total unique tennis players: 2352


2. What is the average height of the players?

In [2]:
# Combine player info from both tables
players_info = pd.concat([MatchHomeTeamInfo[['player_id', 'height']], MatchAwayTeamInfo[['player_id', 'height']]])

# Remove duplicate players
unique_players = players_info.drop_duplicates(subset='player_id')

# Calculate average height
average_height = unique_players['height'].mean()

print(f'The average height of the players is: {average_height:.2f} m')

# fillna#########

The average height of the players is: 1.82 m


3. Which player has the highest number of wins?

In [3]:
MatchEventInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchEventInfo.csv")
MatchEventInfo.head(5)

# Merge MatchEventInfo with MatchHomeTeamInfo and MatchAwayTeamInfo to get player IDs
merged_df = MatchEventInfo[['match_id', 'winner_code']].merge(MatchHomeTeamInfo[['match_id', 'player_id']], on='match_id', suffixes=('', '_home'))
merged_df = merged_df.merge(MatchAwayTeamInfo[['match_id', 'player_id']], on='match_id', suffixes=('', '_away')).drop_duplicates(subset="match_id")
merged_df.head(5)

# Map winner_code to actual player_id
merged_df['winner_id'] = merged_df.apply(
    lambda row: row['player_id'] if row['winner_code'] == 1.0 else row['player_id_away'], 
    axis=1
)

# Count the number of wins for each player
win_counts = merged_df['winner_id'].value_counts()

# Find the player with the highest number of wins
top_player_id = win_counts.idxmax()
top_player_wins = win_counts.max()

print(f'Player with the highest number of wins: {top_player_id}')
print(f'Number of wins: {top_player_wins}')

Player with the highest number of wins: 253356.0
Number of wins: 17


4. What is the longest match recorded in terms of duration?

In [4]:
MatchTimeInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\MatchTimeInfo.csv")
MatchTimeInfo.head(5)

# Calculate the total duration of each match by summing the periods
MatchTimeInfo['total_duration'] = MatchTimeInfo[['period_1', 'period_2', 'period_3', 'period_4', 'period_5']].sum(axis=1)

# Find the match with the longest duration
longest_match = MatchTimeInfo.loc[MatchTimeInfo['total_duration'].idxmax()]

print("Longest match recorded in terms of duration:")
print(f"Match ID: {longest_match['match_id']}")
print(f"Duration: {longest_match['total_duration']}")



Longest match recorded in terms of duration:
Match ID: 12346747.0
Duration: 171153.0


6. Which country has produced the most successful tennis players?

In [5]:
# Combine MatchHomeTeamInfo and MatchAwayTeamInfo to aggregate data
combined_teams = pd.concat([MatchHomeTeamInfo, MatchAwayTeamInfo], ignore_index=True)

# Group by country and sum the total prize money
total_prize_per_country = combined_teams.groupby('country')['total_prize'].sum()

# Find the country with the highest total prize money earned
most_successful_country = total_prize_per_country.idxmax()
max_total_prize = total_prize_per_country.max()

# Print the result
print(f"The country that has produced the most successful tennis players based on total prize money earned is {most_successful_country} with total prize money of ${max_total_prize}.")

The country that has produced the most successful tennis players based on total prize money earned is Serbia with total prize money of $2864742758.0.


8. Is there a difference in the number of double faults based on gender?

In [6]:
PeriodInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\PeriodInfo.csv")

# Filter rows where the statistic_name is "double_faults"
double_faults_data = PeriodInfo[PeriodInfo['statistic_name'] == 'double_faults']

# Merge with MatchHomeTeamInfo to get home team gender and double faults
home_merged = pd.merge(double_faults_data, MatchHomeTeamInfo[['match_id', 'gender']], on='match_id')
home_double_faults = home_merged[['gender', 'home_stat']].rename(columns={'home_stat': 'double_faults'})

# Merge with MatchAwayTeamInfo to get away team gender and double faults
away_merged = pd.merge(double_faults_data, MatchAwayTeamInfo[['match_id', 'gender']], on='match_id')
away_double_faults = away_merged[['gender', 'away_stat']].rename(columns={'away_stat': 'double_faults'})

# Combine home and away double faults
combined_double_faults = pd.concat([home_double_faults, away_double_faults])

# Convert double_faults to numeric
combined_double_faults['double_faults'] = pd.to_numeric(combined_double_faults['double_faults'])

# Group by gender and calculate the total number of double faults
double_faults_by_gender = combined_double_faults.groupby('gender')['double_faults'].sum()

# Calculate total number of females and males
gender_counts = combined_double_faults['gender'].value_counts()

# Calculate the ratio of double faults to the total count for each gender
ratios = double_faults_by_gender / gender_counts

print(ratios)


gender
F    2.155839
M    1.607963
dtype: float64


10. Is there a correlation between a player's height and their ranking?

In [68]:
all_players = pd.concat([MatchHomeTeamInfo, MatchAwayTeamInfo])

# Drop duplicates and keep the last record of each match_id
all_players_drop_dup = all_players.drop_duplicates(subset=['match_id','name'], keep='last')

correlation_coeff = all_players_drop_dup['height'].corr(all_players_drop_dup['current_rank'])

print("Correlation coefficient between height and ranking:", correlation_coeff, "\nThis value indicates a very weak positive correlation between these two variables")


Correlation coefficient between height and ranking: 0.11243439094485426 
This value indicates a very weak positive correlation between these two variables


12. What is the average number of games per set in men's matches compared to women's matches?

In [144]:
GameInfo = pd.read_csv(r"E:\Data Analysis Course\Data analyse\tennis_data_analysis\data\raw\GameInfo.csv")

GameInfo_with_gender = pd.merge(GameInfo, all_players, on='match_id')

# Drop duplicates and keep the last record based on set_id and game_id
GameInfo_with_gender_cleaned = GameInfo_with_gender.sort_values(by=['match_id', 'set_id', 'game_id']).drop_duplicates(subset=['match_id', 'set_id'], keep='last')

GameInfo_with_gender_cleaned=GameInfo_with_gender_cleaned[["match_id" ,"game_id" , "gender" ]]
GameInfo_Male_cleaned = GameInfo_with_gender_cleaned[GameInfo_with_gender_cleaned['gender'] == 'M']
GameInfo_Female_cleaned = GameInfo_with_gender_cleaned[GameInfo_with_gender_cleaned['gender'] == 'F']
avg_games_per_set_men = GameInfo_Male_cleaned["game_id"].mean()
avg_games_per_set_women = GameInfo_Female_cleaned["game_id"].mean()

# Print the results
print(f"Average number of games per set in men's matches: {avg_games_per_set_men}")
print(f"Average number of games per set in women's matches: {avg_games_per_set_women}")

Average number of games per set in men's matches: 9.213686958911639
Average number of games per set in women's matches: 8.931094383323682
